In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 70 #display 70 dpi in Jupyter Notebook, may consider100 dpi 
plt.rcParams['savefig.dpi'] = 300 #define 300 dpi for saving figures

import seaborn as sns
## here are some settings 
sns.set_style('whitegrid')
sns.set(rc={"figure.dpi":70, 'savefig.dpi':300}) #defining dpi setting
sns.set_context('notebook')
sns.set_style("ticks")
from IPython.display import set_matplotlib_formats
from sklearn.model_selection import LeaveOneGroupOut

In [2]:
# Tells matplotlib to display images inline instead of a new window
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from tqdm import tqdm
import wfdb

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated
print("Hello World")
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

Hello World
---Run time is 0.00010080000356538221 seconds ---



In [3]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import PolynomialFeatures

import sklearn.linear_model as skl_lm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.tree import DecisionTreeRegressor 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.tree import export_graphviz
from sklearn.svm import SVC

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import auc
from sklearn.metrics import roc_curve

In [4]:
#data = load_breast_cancer(as_frame = True); 
#df = data.frame; 
#print(df.shape)
# In this block use the code we looked at on 6/30 as a reference to save our rr-intervals as a .
rlist = []
records = 'mit-bih-raw\RECORDS' # Replace the _______ with the name of the records file in your MIT-BIH data folder
with open(records) as rfile: #Then we open the file 
                             #The 'with' command only opens the file while we are in it. Automatically closes the file when we're not
    for record in rfile:  # Then we iterate through the lines in the file
        record = record[0:len(record)-1] # Remove any erronious new line characters at the end ('\n')
        rlist.append(record) # Then build an array with it
        

###### Step 1: Initialize all Arrays
             # Below, replace all of the ___ with the command that declares an array/list
             # hint: https://stackoverflow.com/questions/1514553/how-to-declare-an-array-in-python
samples = [] # will house the samples of all subjects
good_list = [] # will list the names of the subjects we successfully extracted
bad_list = [] # will house the names of the subjects we failed to extract
qrs = [] # will house the indices of R-Peaks for all subjects
atr_label = [] # will house the labels for each rhythm annotation for all subjects
atr_locs = [] # will house the locations corresponding to the rhythm annotation labels


###### Step 2: Extract Information
for x in tqdm(rlist): #this will iterate through te records that we found above
  
    try: # A try statement will run the except statement if for some reason the try commands fail
         # In this case I use the try statement because one of the subjects has no signal data causing failure
         # I then use bad_list and good_list so that all of the indices in rlist match with the arrays we initialized in Step 1, above
        ######################################################
            # Below find the wfdb function that will return the information that is described below 
        # Then replace _____ with the correct function call
        samp = wfdb.rdsamp('mit-bih-raw/'+x) # wfdb._____(file_location) will read the signal & header data and return a 2 value array
            # samp[0] - the signal data is the raw reading from the ecg. Each value is a sample taken.
            # samp[1] - the header data includes things about the signal data such as:
              # samples per section, denoted 'fs'
              # number of signals, denoted 'n_sig'
            
        ######################################################
        samples.append(samp) #add it to our array for all subject
            #What is our file extension that has the annotation we want? Find it here and replace _____ with it 
            #hint: READ THE VARIABLE NAMES!!!!
        qrs_tmp = wfdb.rdann('mit-bih-raw/'+x, extension='qrs') #extract the QRS Info
        qrs_locs = np.array(qrs_tmp.sample, dtype='int') #Get just the loccation of R-Peaks from the QRS Info
        qrs.append(qrs_locs) # Add to our array for all subjects
        
            #Do the same thing here
        atr = wfdb.rdann('mit-bih-raw/'+x,extension='atr') #extract the atr info which stores the rhythm type(s) over the whole signal
        atr_label.append(atr.aux_note) # aux_note stores the type of rhythm - main two are '(N' for normal and '(AFIB' for AFIB
        atr_locs.append(np.append(atr.sample, len(samp[0]))) #I add the length of the whole sample to the end for better visualization later
        
        good_list.append(x) # when all extraction is successful append the record name to good_list
    except Exception as exep:
        print(exep) # Alert the user of an exception
        bad_list.append(x) # add to the bad list


  0%|          | 0/25 [00:00<?, ?it/s]

sampto must be greater than sampfrom
sampto must be greater than sampfrom


100%|██████████| 25/25 [00:16<00:00,  1.48it/s]


In [5]:
features_df_list = []
for subject in good_list:
    data = pd.read_csv(f'subject-features-dataframes/subject{subject}.csv')
    data = data.assign(Subject = subject)
    #print(data)
    features_df_list.append(data)
#features_maindf = pd.concat(features_df_list, ignore_index = True)
features_maindf = pd.concat(features_df_list, ignore_index = True)
print(features_maindf.columns)

Index(['Unnamed: 0', 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
       'AppEn', 'SampEn', 'Rhythm', 'Subject'],
      dtype='object')


In [6]:
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []

df = features_maindf
x_values = ['StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
       'AppEn', 'SampEn']
y_values = ['Rhythm']

#logo = LeaveOneGroupOut()
#splits = list(logo.split(x_values, y_values, groups=df))

### Support Vector Classifier 

The ${\tt SVC()}$ function can be used to fit a
support vector classifier when the argument ${\tt kernel="linear"}$ is used. This
function uses a slightly different formulation of the equations we saw in lecture to build the
support vector classifier. The ${\tt c}$ argument allows us to specify the cost of
a violation to the margin. When the ${\tt c}$ argument is **small**, then the margins
will be wide and many support vectors will be on the margin or will
violate the margin. When the ${\tt c}$ argument is large, then the margins will
be narrow and there will be few support vectors on the margin or violating
the margin.

We can use the ${\tt SVC()}$ function to fit the support vector classifier for a
given value of the ${\tt cost}$ parameter. 

In [7]:
#for df in features_df_list:
#model = DecisionTreeClassifier(max_depth = 6) #criterion='entropy'
df = features_maindf
acc_score = [];
Truth = [];
Output = [];
max_depth_list = []
for idx, subject in tqdm(enumerate(good_list)):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]

#     temp_model = DecisionTreeClassifier(max_depth = 6)    
#     param_distributions = {'max_depth': randint(7,9)}

#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#    max_depth_list.append(search.best_params_['max_depth'])
    
    model = SVC(C=1, kernel='linear', random_state=2, probability = True) #criterion='entropy'
    model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(acc_score)
print()
print(confusion_matrix(Truth, Output))
print(classification_report(Truth, Output))
print()


23it [17:15, 45.01s/it]

---Run time is 1035.1536317999999 seconds ---

Accuracy of each fold: 
 [0.915903890160183, 0.9549695740365112, 0.9962216624685138, 0.9382494004796164, 0.9973876698014629, 0.9456389452332657, 0.8047641289117234, 0.9965963240299524, 0.9437751004016064, 0.9702315325248071, 0.9836660617059891, 0.9913854989231874, 0.7591836734693878, 0.9636942675159236, 0.5678601165695254, 0.9813912272928667, 0.9890335846470185, 0.9844200807847663, 0.7447168216398986, 0.9697636063771303, 0.9944396920444825, 0.997489014438167, 0.9919661733615222]

Avg accuracy : 
0.9296846976877177

Std of accuracy : 
0.10583806544858791
[0.915903890160183, 0.9549695740365112, 0.9962216624685138, 0.9382494004796164, 0.9973876698014629, 0.9456389452332657, 0.8047641289117234, 0.9965963240299524, 0.9437751004016064, 0.9702315325248071, 0.9836660617059891, 0.9913854989231874, 0.7591836734693878, 0.9636942675159236, 0.5678601165695254, 0.9813912272928667, 0.9890335846470185, 0.9844200807847663, 0.7447168216398986, 0.96976360637

### ${\tt sklearn.grid\_search}$ module

The ${\tt sklearn.grid\_search}$ module includes a a function ${\tt GridSearchCV()}$ to perform cross-validation. In order to use this function, we pass in relevant information about the set of models that are under consideration. The
following command indicates that we want perform 10-fold cross-validation to compare SVMs with a linear
kernel, using a range of values of the cost parameter:

In [8]:
from sklearn.model_selection import GridSearchCV

# Select the optimal C parameter by cross-validation
#tuned_parameters = [{'C': [0.001, 0.01, 0.1, 1, 5, 10, 100]}]
tuned_parameters = [{'C': [0.001, 0.01, 1, 10]}]

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

clf = GridSearchCV(SVC(kernel='linear'), tuned_parameters, cv=10, scoring='accuracy')
clf.fit(X_train, y_train)

elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

---Run time is 356.6876246999964 seconds ---



In [9]:
print(clf.best_params_)

{'C': 10}


In [10]:
df = features_maindf
acc_score = [];
Truth = [];
Output = [];
max_depth_list = []
for idx, subject in tqdm(enumerate(good_list)):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]

#     temp_model = DecisionTreeClassifier(max_depth = 6)    
#     param_distributions = {'max_depth': randint(7,9)}

#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     max_depth_list.append(search.best_params_['max_depth'])
    
    model = SVC(C=0.1, kernel='linear', random_state=2, probability = True) #criterion='entropy'
    model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(acc_score)
print()
print(confusion_matrix(Truth, Output))
print(classification_report(Truth, Output))
print()


# crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)
# model = SVC(C=0.01, kernel='linear')
# scores = cross_val_score(model, X, y, cv=crossvalidation)
# print('Accuracy of each fold: \n {}'.format(scores))
# print()
# print("Avg accuracy: {}".format(scores.mean()))

23it [19:42, 51.43s/it]

---Run time is 1539.6610896999991 seconds ---

Accuracy of each fold: 
 [0.9124713958810069, 0.954158215010142, 0.9949622166246851, 0.9166666666666666, 0.9963427377220481, 0.9415821501014199, 0.8510042036431574, 0.9965963240299524, 0.945281124497992, 0.9669239250275634, 0.9841197822141561, 0.9899497487437185, 0.7596371882086168, 0.978343949044586, 0.515820149875104, 0.976517501107665, 0.9897189856065799, 0.986728216964801, 0.7324598478444633, 0.965915338097856, 0.9940119760479041, 0.9949780288763339, 0.9911205073995771]

Avg accuracy : 
0.9276221817059129

Std of accuracy : 
0.11281202828874064
[0.9124713958810069, 0.954158215010142, 0.9949622166246851, 0.9166666666666666, 0.9963427377220481, 0.9415821501014199, 0.8510042036431574, 0.9965963240299524, 0.945281124497992, 0.9669239250275634, 0.9841197822141561, 0.9899497487437185, 0.7596371882086168, 0.978343949044586, 0.515820149875104, 0.976517501107665, 0.9897189856065799, 0.986728216964801, 0.7324598478444633, 0.965915338097856, 0.99

### Support Vector Machine

In order to fit an SVM using a **non-linear kernel**, we once again use the ${\tt SVC()}$
function. However, now we use a different value of the parameter kernel.
To fit an SVM with a polynomial kernel we use ${\tt kernel="poly"}$, and
to fit an SVM with a radial kernel we use ${\tt kernel="rbf"}$. In the former
case we also use the ${\tt degree}$ argument to specify a degree for the polynomial
kernel, and in the latter case we use ${\tt gamma}$ to specify a
value of $\gamma$ for the radial basis kernel.

Let's generate some data with a non-linear class boundary:

In [11]:
# df = features_maindf
# acc_score = [];
# Truth = [];
# Output = [];
# max_depth_list = []
# for idx, subject in tqdm(enumerate(good_list)):
    
#     train_index = ~df['Subject'].str.contains(subject)
#     test_index = df['Subject'].str.contains(subject)
#     X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
#     y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]

# #     temp_model = DecisionTreeClassifier(max_depth = 6)    
# #     param_distributions = {'max_depth': randint(7,9)}

# #     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
# #     search.fit(X_train, y_train)

# #     max_depth_list.append(search.best_params_['max_depth'])
    
#     model = SVC(C=0.01, gamma=1, kernel='rbf')
#     model.fit(X_train,y_train)

#     pred_values = model.predict(X_test)
#     pred_prob = model.predict_proba(X_test)[:, 1]
     
#     acc = accuracy_score(y_test, pred_values)
#     acc_score.append(acc)
    
#     Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
#     Output.extend(pred_values); ## it is a list 
# elapsed = timeit.default_timer() - start_time #gives total computation time
# print("---Run time is %s seconds ---" % elapsed) #prints computation time
# print()

# print('Accuracy of each fold: \n {}'.format(acc_score))
# print()
# print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
# print()
# print('Std of accuracy : \n{}'.format(np.std(acc_score)))

# # crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)
# # model = SVC(C=0.01, gamma=1, kernel='rbf')
# # scores = cross_val_score(model, X, y, cv=crossvalidation)
# # print('Accuracy of each fold: \n {}'.format(scores))
# # print()
# # print("Avg accuracy: {}".format(scores.mean()))

In [12]:
# tuned_parameters = [{'C': [0.01, 0.1, 1, 10, 100],
#                      'gamma': [0.01, 0.1, 0.5, 1, 5, 10]}]

# clf = GridSearchCV(SVC(kernel='rbf'), tuned_parameters, cv=10, scoring='accuracy')
# clf.fit(X_train, y_train)
# clf.best_params_

In [13]:
# df = features_maindf
# acc_score = [];
# Truth = [];
# Output = [];
# max_depth_list = []
# for idx, subject in tqdm(enumerate(good_list)):
    
#     train_index = ~df['Subject'].str.contains(subject)
#     test_index = df['Subject'].str.contains(subject)
#     X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
#     y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]

# #     temp_model = DecisionTreeClassifier(max_depth = 6)    
# #     param_distributions = {'max_depth': randint(7,9)}

# #     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
# #     search.fit(X_train, y_train)

# #     max_depth_list.append(search.best_params_['max_depth'])
    
#     model = SVC(C=0.01, gamma=0.01, kernel='rbf')
#     model.fit(X_train,y_train)

#     pred_values = model.predict(X_test)
#     pred_prob = model.predict_proba(X_test)[:, 1]
     
#     acc = accuracy_score(y_test, pred_values)
#     acc_score.append(acc)
    
#     Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
#     Output.extend(pred_values); ## it is a list 
# elapsed = timeit.default_timer() - start_time #gives total computation time
# print("---Run time is %s seconds ---" % elapsed) #prints computation time
# print()

# print('Accuracy of each fold: \n {}'.format(acc_score))
# print()
# print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
# print()
# print('Std of accuracy : \n{}'.format(np.std(acc_score)))

# # crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)
# # model = SVC(C=0.01, gamma=0.01, kernel='rbf')
# # scores = cross_val_score(model, X, y, cv=crossvalidation)
# # print('Accuracy of each fold: \n {}'.format(scores))
# # print()
# # print("Avg accuracy: {}".format(scores.mean()))